# Lab 2.4: Neural Network Class Design

**Duration:** 45 minutes

## Learning Objectives
By the end of this lab, you will be able to:
- Design and implement a flexible neural network class from scratch
- Understand object-oriented programming principles for ML models
- Implement forward propagation for multi-layer networks
- Create modular, reusable components for neural networks
- Build a foundation for implementing various network architectures
- Apply software engineering best practices to machine learning code

## Prerequisites
- Python 3.8+
- NumPy, Matplotlib
- Understanding of neural network fundamentals
- Completed previous labs (especially Lab 2.1 and 2.3)

## Overview
Creating well-structured, maintainable neural network code is crucial for real-world applications. This lab focuses on designing a robust, flexible neural network class that can handle different architectures, activation functions, and training procedures. You'll learn to build modular components that can be easily extended and modified.

## Step 1: Environment Setup and Design Principles

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from typing import List, Tuple, Optional, Callable, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print("Environment setup complete!")
print(f"NumPy version: {np.__version__}")

print("\nNeural Network Design Principles:")
print("=" * 35)
print("1. Modularity: Separate concerns (layers, activations, optimizers)")
print("2. Flexibility: Support different architectures and hyperparameters")
print("3. Extensibility: Easy to add new components")
print("4. Maintainability: Clean, readable, well-documented code")
print("5. Performance: Efficient NumPy operations")
print("6. Debugging: Built-in logging and diagnostic tools")

## Step 2: Activation Function Components

Let's start by creating a modular activation function system that can be easily extended.

In [ ]:
class ActivationFunction(ABC):
    """
    Abstract base class for activation functions
    Defines the interface that all activation functions must implement
    """
    
    @abstractmethod
    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the forward pass of the activation function
        
        Args:
            x: Input array
        
        Returns:
            Activated output
        """
        pass
    
    @abstractmethod
    def backward(self, x: np.ndarray) -> np.ndarray:
        """
        Compute the derivative of the activation function
        
        Args:
            x: Input array (pre-activation values)
        
        Returns:
            Derivative of activation function
        """
        pass
    
    @property
    @abstractmethod
    def name(self) -> str:
        """
        Return the name of the activation function
        """
        pass


class Sigmoid(ActivationFunction):
    """
    Sigmoid activation function: σ(x) = 1 / (1 + e^(-x))
    """
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        # Clip to prevent overflow
        x_clipped = np.clip(x, -500, 500)
        return 1 / (1 + np.exp(-x_clipped))
    
    def backward(self, x: np.ndarray) -> np.ndarray:
        s = self.forward(x)
        return s * (1 - s)
    
    @property
    def name(self) -> str:
        return "Sigmoid"


class Tanh(ActivationFunction):
    """
    Hyperbolic tangent activation function: tanh(x) = (e^x - e^(-x)) / (e^x + e^(-x))
    """
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.tanh(x)
    
    def backward(self, x: np.ndarray) -> np.ndarray:
        return 1 - np.tanh(x) ** 2
    
    @property
    def name(self) -> str:
        return "Tanh"


class ReLU(ActivationFunction):
    """
    Rectified Linear Unit: ReLU(x) = max(0, x)
    """
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.maximum(0, x)
    
    def backward(self, x: np.ndarray) -> np.ndarray:
        return (x > 0).astype(float)
    
    @property
    def name(self) -> str:
        return "ReLU"


class LeakyReLU(ActivationFunction):
    """
    Leaky ReLU: LeakyReLU(x) = max(αx, x) where α is a small constant
    """
    
    def __init__(self, alpha: float = 0.01):
        self.alpha = alpha
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        return np.where(x > 0, x, self.alpha * x)
    
    def backward(self, x: np.ndarray) -> np.ndarray:
        return np.where(x > 0, 1.0, self.alpha)
    
    @property
    def name(self) -> str:
        return f"LeakyReLU(α={self.alpha})"


class Linear(ActivationFunction):
    """
    Linear activation function: f(x) = x (identity function)
    """
    
    def forward(self, x: np.ndarray) -> np.ndarray:
        return x
    
    def backward(self, x: np.ndarray) -> np.ndarray:
        return np.ones_like(x)
    
    @property
    def name(self) -> str:
        return "Linear"


# Activation function factory
ACTIVATION_FUNCTIONS = {
    'sigmoid': Sigmoid,
    'tanh': Tanh,
    'relu': ReLU,
    'leaky_relu': LeakyReLU,
    'linear': Linear
}


def get_activation(name: str, **kwargs) -> ActivationFunction:
    """
    Factory function to create activation functions
    
    Args:
        name: Name of activation function
        **kwargs: Additional parameters for activation function
    
    Returns:
        Activation function instance
    """
    if name.lower() not in ACTIVATION_FUNCTIONS:
        raise ValueError(f"Unknown activation function: {name}. Available: {list(ACTIVATION_FUNCTIONS.keys())}")
    
    return ACTIVATION_FUNCTIONS[name.lower()](**kwargs)


print("✓ Activation function components implemented successfully!")
print(f"Available activation functions: {list(ACTIVATION_FUNCTIONS.keys())}")

## Step 3: Layer Components

Now let's create a modular layer system that forms the building blocks of our neural networks.

In [ ]:
class Layer:
    """
    Neural network layer with weights, biases, and activation function
    """
    
    def __init__(self, input_size: int, output_size: int, 
                 activation: str = 'sigmoid',
                 weight_init: str = 'xavier',
                 **activation_kwargs):
        """
        Initialize layer
        
        Args:
            input_size: Number of input neurons
            output_size: Number of output neurons
            activation: Name of activation function
            weight_init: Weight initialization method
            **activation_kwargs: Additional activation function parameters
        """
        self.input_size = input_size
        self.output_size = output_size
        self.activation = get_activation(activation, **activation_kwargs)
        
        # Initialize weights and biases
        self.weights = self._initialize_weights(weight_init)
        self.biases = np.zeros((1, output_size))
        
        # Store intermediate values for backpropagation
        self.last_input = None
        self.last_z = None
        self.last_activation = None
        
        # Gradients
        self.dW = None
        self.db = None
    
    def _initialize_weights(self, method: str) -> np.ndarray:
        """
        Initialize weights using specified method
        
        Args:
            method: Initialization method ('xavier', 'he', 'random', 'zeros')
        
        Returns:
            Initialized weight matrix
        """
        if method == 'xavier':
            # Xavier/Glorot initialization
            limit = np.sqrt(6 / (self.input_size + self.output_size))
            return np.random.uniform(-limit, limit, (self.input_size, self.output_size))
        
        elif method == 'he':
            # He initialization (good for ReLU)
            return np.random.normal(0, np.sqrt(2 / self.input_size), 
                                  (self.input_size, self.output_size))
        
        elif method == 'random':
            # Simple random initialization
            return np.random.normal(0, 0.01, (self.input_size, self.output_size))
        
        elif method == 'zeros':
            # Zero initialization (not recommended for hidden layers)
            return np.zeros((self.input_size, self.output_size))
        
        else:
            raise ValueError(f"Unknown weight initialization method: {method}")
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward propagation through the layer
        
        Args:
            X: Input data (batch_size, input_size)
        
        Returns:
            Layer output (batch_size, output_size)
        """
        # Store input for backpropagation
        self.last_input = X
        
        # Linear transformation: Z = XW + b
        self.last_z = np.dot(X, self.weights) + self.biases
        
        # Apply activation function
        self.last_activation = self.activation.forward(self.last_z)
        
        return self.last_activation
    
    def backward(self, dA: np.ndarray) -> np.ndarray:
        """
        Backward propagation through the layer
        
        Args:
            dA: Gradient of loss with respect to layer output
        
        Returns:
            Gradient of loss with respect to layer input
        """
        m = self.last_input.shape[0]
        
        # Gradient of activation function
        dZ = dA * self.activation.backward(self.last_z)
        
        # Gradients of weights and biases
        self.dW = (1/m) * np.dot(self.last_input.T, dZ)
        self.db = (1/m) * np.sum(dZ, axis=0, keepdims=True)
        
        # Gradient with respect to input (for previous layer)
        dA_prev = np.dot(dZ, self.weights.T)
        
        return dA_prev
    
    def update_parameters(self, learning_rate: float):
        """
        Update layer parameters using gradients
        
        Args:
            learning_rate: Learning rate for gradient descent
        """
        self.weights -= learning_rate * self.dW
        self.biases -= learning_rate * self.db
    
    def get_parameters(self) -> Dict[str, np.ndarray]:
        """
        Get layer parameters
        
        Returns:
            Dictionary containing weights and biases
        """
        return {
            'weights': self.weights.copy(),
            'biases': self.biases.copy()
        }
    
    def set_parameters(self, params: Dict[str, np.ndarray]):
        """
        Set layer parameters
        
        Args:
            params: Dictionary containing weights and biases
        """
        self.weights = params['weights'].copy()
        self.biases = params['biases'].copy()
    
    def __repr__(self) -> str:
        return (f"Layer(input={self.input_size}, output={self.output_size}, "
                f"activation={self.activation.name})")


print("✓ Layer components implemented successfully!")

## Step 4: Loss Function Components

Let's create a modular loss function system for different types of problems.

In [ ]:
class LossFunction(ABC):
    """
    Abstract base class for loss functions
    """
    
    @abstractmethod
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        """
        Compute the loss
        
        Args:
            y_pred: Predicted values
            y_true: True values
        
        Returns:
            Loss value
        """
        pass
    
    @abstractmethod
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        """
        Compute the gradient of loss with respect to predictions
        
        Args:
            y_pred: Predicted values
            y_true: True values
        
        Returns:
            Gradient of loss
        """
        pass
    
    @property
    @abstractmethod
    def name(self) -> str:
        """Return the name of the loss function"""
        pass


class MeanSquaredError(LossFunction):
    """
    Mean Squared Error loss: MSE = (1/2m) * Σ(y_pred - y_true)²
    """
    
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        m = y_true.shape[0]
        return (1/(2*m)) * np.sum((y_pred - y_true) ** 2)
    
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        m = y_true.shape[0]
        return (1/m) * (y_pred - y_true)
    
    @property
    def name(self) -> str:
        return "Mean Squared Error"


class BinaryCrossEntropy(LossFunction):
    """
    Binary Cross Entropy loss: BCE = -1/m * Σ[y*log(p) + (1-y)*log(1-p)]
    """
    
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        m = y_true.shape[0]
        
        # Clip predictions to prevent log(0)
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        
        return -1/m * np.sum(y_true * np.log(y_pred_clipped) + 
                           (1 - y_true) * np.log(1 - y_pred_clipped))
    
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        m = y_true.shape[0]
        
        # Clip predictions to prevent division by zero
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        
        return -1/m * (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
    
    @property
    def name(self) -> str:
        return "Binary Cross Entropy"


class MeanAbsoluteError(LossFunction):
    """
    Mean Absolute Error loss: MAE = 1/m * Σ|y_pred - y_true|
    """
    
    def forward(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        m = y_true.shape[0]
        return (1/m) * np.sum(np.abs(y_pred - y_true))
    
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray) -> np.ndarray:
        m = y_true.shape[0]
        return (1/m) * np.sign(y_pred - y_true)
    
    @property
    def name(self) -> str:
        return "Mean Absolute Error"


# Loss function factory
LOSS_FUNCTIONS = {
    'mse': MeanSquaredError,
    'binary_crossentropy': BinaryCrossEntropy,
    'mae': MeanAbsoluteError
}


def get_loss_function(name: str) -> LossFunction:
    """
    Factory function to create loss functions
    
    Args:
        name: Name of loss function
    
    Returns:
        Loss function instance
    """
    if name.lower() not in LOSS_FUNCTIONS:
        raise ValueError(f"Unknown loss function: {name}. Available: {list(LOSS_FUNCTIONS.keys())}")
    
    return LOSS_FUNCTIONS[name.lower()]()


print("✓ Loss function components implemented successfully!")
print(f"Available loss functions: {list(LOSS_FUNCTIONS.keys())}")

## Step 5: Neural Network Class Design

Now let's create the main neural network class that ties everything together.

In [ ]:
class NeuralNetwork:
    """
    Flexible Neural Network class supporting multiple layers and configurations
    """
    
    def __init__(self, architecture: List[Dict[str, Any]], 
                 loss_function: str = 'mse',
                 learning_rate: float = 0.01,
                 random_state: Optional[int] = None):
        """
        Initialize neural network
        
        Args:
            architecture: List of layer configurations
                Example: [
                    {'input_size': 4, 'output_size': 8, 'activation': 'relu'},
                    {'input_size': 8, 'output_size': 4, 'activation': 'relu'},
                    {'input_size': 4, 'output_size': 1, 'activation': 'sigmoid'}
                ]
            loss_function: Name of loss function to use
            learning_rate: Learning rate for gradient descent
            random_state: Random seed for reproducibility
        """
        if random_state is not None:
            np.random.seed(random_state)
        
        self.architecture = architecture
        self.learning_rate = learning_rate
        self.loss_function = get_loss_function(loss_function)
        
        # Build network layers
        self.layers = self._build_layers()
        
        # Training history
        self.training_history = {
            'loss': [],
            'accuracy': []
        }
        
        # Network properties
        self.is_trained = False
        self.n_parameters = self._count_parameters()
    
    def _build_layers(self) -> List[Layer]:
        """
        Build network layers from architecture specification
        
        Returns:
            List of Layer objects
        """
        layers = []
        
        for layer_config in self.architecture:
            # Extract layer parameters
            input_size = layer_config['input_size']
            output_size = layer_config['output_size']
            activation = layer_config.get('activation', 'sigmoid')
            weight_init = layer_config.get('weight_init', 'xavier')
            
            # Extract activation function parameters
            activation_kwargs = {}
            if 'activation_params' in layer_config:
                activation_kwargs = layer_config['activation_params']
            
            # Create layer
            layer = Layer(
                input_size=input_size,
                output_size=output_size,
                activation=activation,
                weight_init=weight_init,
                **activation_kwargs
            )
            
            layers.append(layer)
        
        return layers
    
    def _count_parameters(self) -> int:
        """
        Count total number of trainable parameters
        
        Returns:
            Total number of parameters
        """
        total_params = 0
        for layer in self.layers:
            # Weights + biases
            total_params += layer.weights.size + layer.biases.size
        return total_params
    
    def forward(self, X: np.ndarray) -> np.ndarray:
        """
        Forward propagation through the entire network
        
        Args:
            X: Input data (batch_size, input_features)
        
        Returns:
            Network output (batch_size, output_features)
        """
        activation = X
        
        # Pass through each layer
        for layer in self.layers:
            activation = layer.forward(activation)
        
        return activation
    
    def backward(self, y_pred: np.ndarray, y_true: np.ndarray):
        """
        Backward propagation through the entire network
        
        Args:
            y_pred: Network predictions
            y_true: True target values
        """
        # Compute loss gradient
        dA = self.loss_function.backward(y_pred, y_true)
        
        # Backpropagate through layers (in reverse order)
        for layer in reversed(self.layers):
            dA = layer.backward(dA)
    
    def update_parameters(self):
        """
        Update all layer parameters using computed gradients
        """
        for layer in self.layers:
            layer.update_parameters(self.learning_rate)
    
    def train_step(self, X: np.ndarray, y: np.ndarray) -> Tuple[float, float]:
        """
        Perform one training step (forward + backward + update)
        
        Args:
            X: Input features
            y: Target values
        
        Returns:
            Tuple of (loss, accuracy)
        """
        # Forward propagation
        y_pred = self.forward(X)
        
        # Compute loss
        loss = self.loss_function.forward(y_pred, y)
        
        # Backward propagation
        self.backward(y_pred, y)
        
        # Update parameters
        self.update_parameters()
        
        # Calculate accuracy
        accuracy = self._calculate_accuracy(y_pred, y)
        
        return loss, accuracy
    
    def _calculate_accuracy(self, y_pred: np.ndarray, y_true: np.ndarray) -> float:
        """
        Calculate accuracy based on problem type
        
        Args:
            y_pred: Predicted values
            y_true: True values
        
        Returns:
            Accuracy score
        """
        if self.loss_function.name == 'Binary Cross Entropy':
            # Binary classification accuracy
            predictions = (y_pred > 0.5).astype(int)
            return np.mean(predictions == y_true)
        else:
            # For regression, we'll use R² score as "accuracy"
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            r2 = 1 - (ss_res / ss_tot)
            return max(0, r2)  # Return 0 if R² is negative
    
    def fit(self, X: np.ndarray, y: np.ndarray, 
            epochs: int = 1000,
            batch_size: Optional[int] = None,
            validation_data: Optional[Tuple[np.ndarray, np.ndarray]] = None,
            verbose: int = 1) -> Dict[str, List[float]]:
        """
        Train the neural network
        
        Args:
            X: Training features
            y: Training targets
            epochs: Number of training epochs
            batch_size: Batch size (None = full batch)
            validation_data: Validation data tuple (X_val, y_val)
            verbose: Verbosity level (0=silent, 1=progress bar, 2=detailed)
        
        Returns:
            Training history dictionary
        """
        # Ensure correct shapes
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        
        # Initialize history
        self.training_history = {
            'loss': [],
            'accuracy': [],
            'val_loss': [],
            'val_accuracy': []
        }
        
        n_samples = X.shape[0]
        
        # Training loop
        for epoch in range(epochs):
            epoch_loss = 0
            epoch_accuracy = 0
            n_batches = 0
            
            # Batch training
            if batch_size is None or batch_size >= n_samples:
                # Full batch
                loss, accuracy = self.train_step(X, y)
                epoch_loss = loss
                epoch_accuracy = accuracy
                n_batches = 1
            else:
                # Mini-batch training
                indices = np.random.permutation(n_samples)
                
                for i in range(0, n_samples, batch_size):
                    batch_indices = indices[i:i+batch_size]
                    X_batch = X[batch_indices]
                    y_batch = y[batch_indices]
                    
                    loss, accuracy = self.train_step(X_batch, y_batch)
                    epoch_loss += loss
                    epoch_accuracy += accuracy
                    n_batches += 1
                
                epoch_loss /= n_batches
                epoch_accuracy /= n_batches
            
            # Record training metrics
            self.training_history['loss'].append(epoch_loss)
            self.training_history['accuracy'].append(epoch_accuracy)
            
            # Validation metrics
            if validation_data is not None:
                X_val, y_val = validation_data
                if y_val.ndim == 1:
                    y_val = y_val.reshape(-1, 1)
                
                y_val_pred = self.predict(X_val)
                val_loss = self.loss_function.forward(y_val_pred, y_val)
                val_accuracy = self._calculate_accuracy(y_val_pred, y_val)
                
                self.training_history['val_loss'].append(val_loss)
                self.training_history['val_accuracy'].append(val_accuracy)
            
            # Print progress
            if verbose > 0 and (epoch + 1) % max(1, epochs // 10) == 0:
                print(f"Epoch {epoch + 1}/{epochs}:")
                print(f"  Loss: {epoch_loss:.6f}, Accuracy: {epoch_accuracy:.4f}")
                
                if validation_data is not None:
                    print(f"  Val Loss: {val_loss:.6f}, Val Accuracy: {val_accuracy:.4f}")
                print()
        
        self.is_trained = True
        return self.training_history
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Make predictions on new data
        
        Args:
            X: Input features
        
        Returns:
            Predictions
        """
        return self.forward(X)
    
    def evaluate(self, X: np.ndarray, y: np.ndarray) -> Dict[str, float]:
        """
        Evaluate the network on test data
        
        Args:
            X: Test features
            y: Test targets
        
        Returns:
            Dictionary of evaluation metrics
        """
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        
        y_pred = self.predict(X)
        loss = self.loss_function.forward(y_pred, y)
        accuracy = self._calculate_accuracy(y_pred, y)
        
        return {
            'loss': loss,
            'accuracy': accuracy
        }
    
    def summary(self):
        """
        Print network architecture summary
        """
        print("Neural Network Architecture Summary")
        print("=" * 50)
        print(f"Loss Function: {self.loss_function.name}")
        print(f"Learning Rate: {self.learning_rate}")
        print(f"Total Parameters: {self.n_parameters:,}")
        print(f"Trained: {'Yes' if self.is_trained else 'No'}")
        print("\nLayer Architecture:")
        print("-" * 30)
        
        total_params = 0
        for i, layer in enumerate(self.layers):
            layer_params = layer.weights.size + layer.biases.size
            total_params += layer_params
            
            print(f"Layer {i + 1}: {layer.input_size:3d} → {layer.output_size:3d} "
                  f"({layer.activation.name:<10}) [{layer_params:,} params]")
        
        print("-" * 30)
        print(f"Total: {total_params:,} parameters")
        
        if self.is_trained:
            print(f"\nTraining Results:")
            print(f"Final Loss: {self.training_history['loss'][-1]:.6f}")
            print(f"Final Accuracy: {self.training_history['accuracy'][-1]:.4f}")


print("✓ Neural Network class implemented successfully!")

## Step 6: Test the Neural Network Design

Let's test our neural network design with different architectures and problems.

In [ ]:
# Test 1: Simple Binary Classification
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("Test 1: Binary Classification with Simple Architecture")
print("=" * 55)

# Generate dataset
X, y = make_classification(
    n_samples=500,
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_clusters_per_class=1,
    random_state=42
)

# Split and scale
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Dataset: {X_train_scaled.shape[0]} training, {X_test_scaled.shape[0]} test samples")
print(f"Features: {X_train_scaled.shape[1]}, Classes: {len(np.unique(y))}")

# Define network architecture
architecture_simple = [
    {'input_size': 4, 'output_size': 8, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 8, 'output_size': 4, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 4, 'output_size': 1, 'activation': 'sigmoid', 'weight_init': 'xavier'}
]

# Create and train network
nn_simple = NeuralNetwork(
    architecture=architecture_simple,
    loss_function='binary_crossentropy',
    learning_rate=0.01,
    random_state=42
)

# Display architecture
nn_simple.summary()
print("\nTraining...")

# Train the network
history = nn_simple.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test_scaled, y_test),
    verbose=1
)

# Evaluate
train_metrics = nn_simple.evaluate(X_train_scaled, y_train)
test_metrics = nn_simple.evaluate(X_test_scaled, y_test)

print(f"\nFinal Results:")
print(f"Training - Loss: {train_metrics['loss']:.6f}, Accuracy: {train_metrics['accuracy']:.4f}")
print(f"Test - Loss: {test_metrics['loss']:.6f}, Accuracy: {test_metrics['accuracy']:.4f}")

print("\n✓ Test 1 completed successfully!")

In [ ]:
# Test 2: Regression with Deep Architecture
from sklearn.datasets import make_regression

print("\nTest 2: Regression with Deep Architecture")
print("=" * 45)

# Generate regression dataset
X_reg, y_reg = make_regression(
    n_samples=400,
    n_features=6,
    noise=0.1,
    random_state=42
)

# Split and scale
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# Scale targets too for better training
y_scaler = StandardScaler()
y_train_reg_scaled = y_scaler.fit_transform(y_train_reg.reshape(-1, 1)).ravel()
y_test_reg_scaled = y_scaler.transform(y_test_reg.reshape(-1, 1)).ravel()

print(f"Dataset: {X_train_reg_scaled.shape[0]} training, {X_test_reg_scaled.shape[0]} test samples")
print(f"Features: {X_train_reg_scaled.shape[1]}")

# Define deeper architecture
architecture_deep = [
    {'input_size': 6, 'output_size': 16, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 16, 'output_size': 12, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 12, 'output_size': 8, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 8, 'output_size': 4, 'activation': 'relu', 'weight_init': 'he'},
    {'input_size': 4, 'output_size': 1, 'activation': 'linear', 'weight_init': 'xavier'}
]

# Create and train network
nn_deep = NeuralNetwork(
    architecture=architecture_deep,
    loss_function='mse',
    learning_rate=0.001,  # Lower learning rate for regression
    random_state=42
)

# Display architecture
nn_deep.summary()
print("\nTraining...")

# Train the network
history_deep = nn_deep.fit(
    X_train_reg_scaled, y_train_reg_scaled,
    epochs=300,
    batch_size=16,
    validation_data=(X_test_reg_scaled, y_test_reg_scaled),
    verbose=1
)

# Evaluate
train_metrics_reg = nn_deep.evaluate(X_train_reg_scaled, y_train_reg_scaled)
test_metrics_reg = nn_deep.evaluate(X_test_reg_scaled, y_test_reg_scaled)

print(f"\nFinal Results:")
print(f"Training - Loss: {train_metrics_reg['loss']:.6f}, R²: {train_metrics_reg['accuracy']:.4f}")
print(f"Test - Loss: {test_metrics_reg['loss']:.6f}, R²: {test_metrics_reg['accuracy']:.4f}")

print("\n✓ Test 2 completed successfully!")

## Step 7: Visualize Network Performance and Architecture

Let's create visualizations to analyze our neural network designs.

In [ ]:
# Visualization function for training history
def plot_training_history(history: Dict[str, List[float]], title: str = "Training History"):
    """
    Plot training and validation metrics
    
    Args:
        history: Training history dictionary
        title: Plot title
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot loss
    axes[0].plot(history['loss'], 'b-', linewidth=2, label='Training Loss')
    if 'val_loss' in history and history['val_loss']:
        axes[0].plot(history['val_loss'], 'r-', linewidth=2, label='Validation Loss')
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Model Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_yscale('log')
    
    # Plot accuracy
    axes[1].plot(history['accuracy'], 'b-', linewidth=2, label='Training Accuracy')
    if 'val_accuracy' in history and history['val_accuracy']:
        axes[1].plot(history['val_accuracy'], 'r-', linewidth=2, label='Validation Accuracy')
    
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy/R²')
    axes[1].set_title('Model Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# Plot training histories
print("Training History Visualizations")
print("=" * 35)

plot_training_history(history, "Binary Classification Network")
plot_training_history(history_deep, "Deep Regression Network")

print("✓ Training history visualizations completed!")

In [ ]:
# Visualize network architecture
def visualize_network_architecture(network: NeuralNetwork, title: str = "Network Architecture"):
    """
    Visualize network architecture
    
    Args:
        network: Neural network instance
        title: Plot title
    """
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Architecture diagram
    ax1 = axes[0]
    layer_sizes = []
    activation_names = []
    
    for i, layer in enumerate(network.layers):
        if i == 0:
            layer_sizes.append(layer.input_size)
        layer_sizes.append(layer.output_size)
        activation_names.append(layer.activation.name)
    
    # Plot layer sizes as bars
    x_pos = np.arange(len(layer_sizes))
    bars = ax1.bar(x_pos, layer_sizes, alpha=0.7, color=['lightblue' if i == 0 or i == len(layer_sizes)-1 else 'lightcoral' for i in range(len(layer_sizes))])
    
    # Add labels
    ax1.set_xlabel('Layer')
    ax1.set_ylabel('Number of Neurons')
    ax1.set_title('Layer Sizes')
    ax1.set_xticks(x_pos)
    layer_labels = ['Input'] + [f'Hidden {i}' for i in range(1, len(layer_sizes)-1)] + ['Output']
    ax1.set_xticklabels(layer_labels, rotation=45)
    
    # Add values on bars
    for bar, size in zip(bars, layer_sizes):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{size}', ha='center', va='bottom')
    
    ax1.grid(True, alpha=0.3)
    
    # Parameter distribution
    ax2 = axes[1]
    param_counts = []
    layer_names = []
    
    for i, layer in enumerate(network.layers):
        param_count = layer.weights.size + layer.biases.size
        param_counts.append(param_count)
        layer_names.append(f'Layer {i+1}\n({layer.activation.name})')
    
    # Create pie chart for parameter distribution
    ax2.pie(param_counts, labels=layer_names, autopct='%1.1f%%', startangle=90)
    ax2.set_title('Parameter Distribution')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Print detailed layer information
    print(f"\nDetailed Layer Information for {title}:")
    print("-" * 50)
    for i, layer in enumerate(network.layers):
        param_count = layer.weights.size + layer.biases.size
        print(f"Layer {i+1}: {layer.input_size} → {layer.output_size}")
        print(f"  Activation: {layer.activation.name}")
        print(f"  Parameters: {param_count:,} (Weights: {layer.weights.size:,}, Biases: {layer.biases.size})")
        print(f"  Weight shape: {layer.weights.shape}")
        print(f"  Weight range: [{layer.weights.min():.4f}, {layer.weights.max():.4f}]")
        print()

# Visualize both networks
print("Network Architecture Visualizations")
print("=" * 40)

visualize_network_architecture(nn_simple, "Binary Classification Network")
visualize_network_architecture(nn_deep, "Deep Regression Network")

print("\n✓ Architecture visualizations completed!")

## Step 8: Test Different Activation Functions and Architectures

Let's compare how different activation functions and architectures perform.

In [ ]:
# Activation function comparison
print("Activation Function Comparison")
print("=" * 35)

# Create a simple dataset for comparison
X_comp, y_comp = make_classification(
    n_samples=300,
    n_features=3,
    n_redundant=0,
    n_informative=3,
    n_clusters_per_class=1,
    random_state=42
)

X_train_comp, X_test_comp, y_train_comp, y_test_comp = train_test_split(
    X_comp, y_comp, test_size=0.2, random_state=42
)

scaler_comp = StandardScaler()
X_train_comp_scaled = scaler_comp.fit_transform(X_train_comp)
X_test_comp_scaled = scaler_comp.transform(X_test_comp)

# Test different activation functions
activations_to_test = ['sigmoid', 'tanh', 'relu', 'leaky_relu']
activation_results = {}

for activation in activations_to_test:
    print(f"\nTesting {activation} activation...")
    
    # Define architecture with current activation
    architecture_test = [
        {'input_size': 3, 'output_size': 6, 'activation': activation, 'weight_init': 'he' if activation == 'relu' else 'xavier'},
        {'input_size': 6, 'output_size': 4, 'activation': activation, 'weight_init': 'he' if activation == 'relu' else 'xavier'},
        {'input_size': 4, 'output_size': 1, 'activation': 'sigmoid', 'weight_init': 'xavier'}
    ]
    
    # Create and train network
    nn_test = NeuralNetwork(
        architecture=architecture_test,
        loss_function='binary_crossentropy',
        learning_rate=0.01,
        random_state=42
    )
    
    # Train with reduced epochs for comparison
    history_test = nn_test.fit(
        X_train_comp_scaled, y_train_comp,
        epochs=150,
        batch_size=32,
        validation_data=(X_test_comp_scaled, y_test_comp),
        verbose=0
    )
    
    # Evaluate
    test_metrics = nn_test.evaluate(X_test_comp_scaled, y_test_comp)
    
    activation_results[activation] = {
        'network': nn_test,
        'history': history_test,
        'test_accuracy': test_metrics['accuracy'],
        'test_loss': test_metrics['loss'],
        'final_train_loss': history_test['loss'][-1],
        'final_train_accuracy': history_test['accuracy'][-1]
    }
    
    print(f"  Test Accuracy: {test_metrics['accuracy']:.4f}")
    print(f"  Test Loss: {test_metrics['loss']:.6f}")

# Compare results
print("\nActivation Function Comparison Results:")
print("=" * 45)
print(f"{'Activation':<12} {'Test Acc':<10} {'Test Loss':<12} {'Train Acc':<10} {'Train Loss':<12}")
print("-" * 60)

for activation, results in activation_results.items():
    print(f"{activation:<12} {results['test_accuracy']:<10.4f} {results['test_loss']:<12.6f} "
          f"{results['final_train_accuracy']:<10.4f} {results['final_train_loss']:<12.6f}")

print("\n✓ Activation function comparison completed!")

In [ ]:
# Plot activation function comparison
plt.figure(figsize=(15, 10))

# Training loss comparison
plt.subplot(2, 2, 1)
for activation, results in activation_results.items():
    plt.plot(results['history']['loss'], label=f'{activation.title()}', linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss by Activation Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Training accuracy comparison
plt.subplot(2, 2, 2)
for activation, results in activation_results.items():
    plt.plot(results['history']['accuracy'], label=f'{activation.title()}', linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.title('Training Accuracy by Activation Function')
plt.legend()
plt.grid(True, alpha=0.3)

# Final test performance comparison
plt.subplot(2, 2, 3)
activations = list(activation_results.keys())
test_accuracies = [activation_results[act]['test_accuracy'] for act in activations]
test_losses = [activation_results[act]['test_loss'] for act in activations]

x_pos = np.arange(len(activations))
plt.bar(x_pos, test_accuracies, alpha=0.7, color='skyblue')
plt.xlabel('Activation Function')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracy Comparison')
plt.xticks(x_pos, [act.title() for act in activations])
plt.grid(True, alpha=0.3)

# Add values on bars
for i, acc in enumerate(test_accuracies):
    plt.text(i, acc + 0.01, f'{acc:.3f}', ha='center', va='bottom')

# Test loss comparison
plt.subplot(2, 2, 4)
plt.bar(x_pos, test_losses, alpha=0.7, color='lightcoral')
plt.xlabel('Activation Function')
plt.ylabel('Test Loss')
plt.title('Test Loss Comparison')
plt.xticks(x_pos, [act.title() for act in activations])
plt.grid(True, alpha=0.3)

# Add values on bars
for i, loss in enumerate(test_losses):
    plt.text(i, loss + max(test_losses) * 0.02, f'{loss:.3f}', ha='center', va='bottom')

plt.suptitle('Activation Function Performance Comparison', fontsize=16)
plt.tight_layout()
plt.show()

print("\n✓ Activation function comparison plots completed!")

## Progress Tracking Checklist

Mark each completed section:

- [ ] ✅ **Environment Setup**: Imported libraries and established design principles
- [ ] ✅ **Activation Functions**: Built modular activation function system with abstract base class
- [ ] ✅ **Layer Components**: Implemented flexible Layer class with forward/backward propagation
- [ ] ✅ **Loss Functions**: Created modular loss function system for different problem types
- [ ] ✅ **Neural Network Class**: Built comprehensive NeuralNetwork class with training capabilities
- [ ] ✅ **Binary Classification Test**: Successfully tested network on classification problem
- [ ] ✅ **Regression Test**: Validated network performance on regression problem
- [ ] ✅ **Visualization Tools**: Created functions to visualize training and architecture
- [ ] ✅ **Activation Comparison**: Compared different activation functions systematically
- [ ] ✅ **Performance Analysis**: Analyzed and visualized network performance metrics

## Key Concepts Summary

**Object-Oriented Design Principles:**
1. **Abstraction**: Base classes define interfaces for components
2. **Encapsulation**: Components manage their own state and behavior
3. **Modularity**: Separate classes for layers, activations, and loss functions
4. **Extensibility**: Easy to add new components without changing existing code

**Neural Network Architecture:**
- **Layers**: Basic building blocks with weights, biases, and activation functions
- **Forward Pass**: Data flows through layers sequentially
- **Backward Pass**: Gradients propagate back through layers for training
- **Parameter Updates**: Weights and biases updated using computed gradients

**Design Benefits:**
- **Flexibility**: Support for different architectures and hyperparameters
- **Reusability**: Components can be reused across different networks
- **Maintainability**: Clean, well-structured code that's easy to debug
- **Scalability**: Architecture can be easily extended for complex networks

**Best Practices Implemented:**
- Type hints for better code documentation
- Abstract base classes for consistent interfaces
- Factory functions for component creation
- Comprehensive error handling and validation
- Built-in visualization and debugging tools

## Troubleshooting

**Common Issues and Solutions:**

1. **Architecture Specification Errors**
   - *Problem*: Incorrect layer dimensions or missing parameters
   - *Solution*: Validate architecture specification, ensure input/output sizes match

2. **Training Instability**
   - *Problem*: Loss explodes or vanishes during training
   - *Solutions*: Adjust learning rate, use appropriate weight initialization, check activation functions

3. **Memory Issues**
   - *Problem*: Large networks consume too much memory
   - *Solutions*: Use smaller batch sizes, reduce network size, implement gradient checkpointing

4. **Slow Convergence**
   - *Problem*: Network takes too long to train
   - *Solutions*: Increase learning rate, use better weight initialization, normalize inputs

5. **Poor Performance**
   - *Problem*: Network doesn't achieve good accuracy
   - *Solutions*: Try different architectures, activation functions, or preprocessing steps

6. **Gradient Issues**
   - *Problem*: Vanishing or exploding gradients
   - *Solutions*: Use appropriate activation functions, gradient clipping, better initialization

## Validation Steps

**Verify Your Implementation:**
1. ✅ Network architecture builds correctly from specification
2. ✅ Forward propagation produces expected output shapes
3. ✅ Backward propagation computes gradients without errors
4. ✅ Training reduces loss consistently
5. ✅ Different activation functions work as expected
6. ✅ Network generalizes well to test data

## Cleanup Instructions

**After Lab Completion:**
1. Save your notebook with all implementations and results
2. Clear large variables if memory is limited:
   ```python
   # Uncomment if needed
   # del nn_simple, nn_deep, activation_results
   # del X_train_scaled, X_test_scaled
   ```
3. Close plots: `plt.close('all')`

## Next Steps

Your neural network class design is now ready for advanced applications! This foundation enables:
- **Complex Architectures**: Build deeper networks, CNNs, RNNs
- **Advanced Optimizers**: Implement Adam, RMSprop, momentum
- **Regularization**: Add dropout, batch normalization, L1/L2 regularization
- **Custom Components**: Create specialized layers and loss functions
- **Production Deployment**: Scale to real-world applications

The modular design you've created follows industry best practices and will serve as a solid foundation for more advanced deep learning implementations.

**Congratulations! You've completed Lab 2.4 - Neural Network Class Design!** 🎉